In [37]:
class Node:
    """
    Represents a spatial polygon as a graph node.
    """
    def __init__(self, node_id):
        self.node_id = node_id
        self.neighbors = []

    def add_neighbor(self, neighbor_id):
        """
        Adds a neighbor if it's not already in the list.
        """
        if neighbor_id not in self.neighbors and neighbor_id != self.node_id:
            self.neighbors.append(neighbor_id)

    def connectivity(self):
        """
        Returns the number of neighbors.
        """
        return len(self.neighbors)

    def __repr__(self):
        return f"Node({self.node_id}, neighbors={self.neighbors})"

In [39]:
class Graph:
    """
    Represents an adjacency graph of spatial polygons.
    """
    def __init__(self, gal_file):
        self.nodes = {}
        self.read_gal(gal_file)

    def read_gal(self, file_path):
        """
        Reads a GAL file and builds the adjacency graph.
        """
        with open(file_path, 'r') as f:
            lines = f.readlines()

        i = 0
        while i < len(lines):
            parts = lines[i].strip().split()
            if len(parts) < 2:
                i += 1
                continue

            node_id = int(parts[0])
            num_neighbors = int(parts[1])

            if node_id not in self.nodes:
                self.nodes[node_id] = Node(node_id)

            if num_neighbors > 0 and i + 1 < len(lines):
                neighbors = list(map(int, lines[i + 1].strip().split()))
                for neighbor in neighbors:
                    if neighbor not in self.nodes:
                        self.nodes[neighbor] = Node(neighbor)
                    self.nodes[node_id].add_neighbor(neighbor)
                    self.nodes[neighbor].add_neighbor(node_id)

            i += 2

    def average_connectivity(self):
        """
        Returns the average number of neighbors per node.
        """
        total_neighbors = sum(node.connectivity() for node in self.nodes.values())
        return total_neighbors / len(self.nodes) if self.nodes else 0

    def max_connectivity(self):
        """
        Returns the highest number of neighbors and the corresponding nodes.
        """
        max_value = max(node.connectivity() for node in self.nodes.values())
        max_nodes = [node for node in self.nodes.values() if node.connectivity() == max_value]
        return max_value, max_nodes
        
    def min_connectivity(self):
        """
        Returns the minimum connectivity (excluding isolated nodes) and the corresponding nodes.
        """
        connected_nodes = [node for node in self.nodes.values() if node.connectivity() > 0]

        if not connected_nodes:
            return 0, []

        min_value = min(node.connectivity() for node in connected_nodes)
        min_nodes = [node for node in connected_nodes if node.connectivity() == min_value]
        return min_value, min_nodes

    def disconnected_nodes(self):
        """
        Returns a list of nodes with no neighbors (isolated nodes).
        """
        return [node for node in self.nodes.values() if node.connectivity() == 0]

    def summary(self):
        """
        Prints a summary of the graph statistics.
        """
        print("\nGraph Summary:")
        print(f"- Average connectivity: {self.average_connectivity():.2f}")
        
        max_value, max_nodes = self.max_connectivity()
        print(f"- Maximum connectivity: {max_value}, Nodes: {max_nodes}")

        min_value, min_nodes = self.min_connectivity()
        print(f"- Minimum connectivity: {min_value}, Nodes: {min_nodes}")

        isolated = self.disconnected_nodes()
        if isolated:
            print(f"- Disconnected nodes: {isolated}")
        else:
            print("- No disconnected nodes.")

In [41]:
import os
os.chdir("/Users/stuber/Desktop/UA/Lab04")

if __name__ == "__main__":
    file_path = "Lab04-1.gal"
    graph = Graph(file_path)
    graph.summary()


Graph Summary:
- Average connectivity: 3.64
- Maximum connectivity: 17, Nodes: [Node(4, neighbors=[3, 5, 20, 19, 23, 35, 29, 28, 38, 37, 36, 41, 45, 46, 48, 49, 40])]
- Minimum connectivity: 1, Nodes: [Node(1, neighbors=[0]), Node(10, neighbors=[26]), Node(11, neighbors=[20]), Node(13, neighbors=[16]), Node(17, neighbors=[25]), Node(21, neighbors=[35]), Node(22, neighbors=[34]), Node(27, neighbors=[28]), Node(31, neighbors=[37]), Node(39, neighbors=[43])]
- No disconnected nodes.
